In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

age  affordability  have_insurance
0   22              1               0
1   25              0               0
2   47              1               1
3   52              0               0
4   46              1               1

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordability']],df.have_insurance,test_size=0.2, random_state=25)

In [4]:
X_train

age  affordability
0    22              1
13   29              0
6    55              0
17   58              1
24   50              1
19   18              1
25   54              1
16   25              0
20   21              1
3    52              0
7    60              0
1    25              0
5    56              1
26   23              1
8    62              1
18   19              0
12   27              0
23   45              1
22   40              1
15   55              1
4    46              1

In [5]:
len(X_train)

21

In [6]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [7]:
X_train_scaled

age  affordability
0   0.22              1
13  0.29              0
6   0.55              0
17  0.58              1
24  0.50              1
19  0.18              1
25  0.54              1
16  0.25              0
20  0.21              1
3   0.52              0
7   0.60              0
1   0.25              0
5   0.56              1
26  0.23              1
8   0.62              1
18  0.19              0
12  0.27              0
23  0.45              1
22  0.40              1
15  0.55              1
4   0.46              1

In [8]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 9ms/step - loss: 0.5235 - accuracy: 0.6190
Epoch 2502/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5235 - accuracy: 0.6190
Epoch 2503/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5234 - accuracy: 0.6190
Epoch 2504/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5234 - accuracy: 0.6190
Epoch 2505/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5234 - accuracy: 0.6190
Epoch 2506/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5234 - accuracy: 0.6190
Epoch 2507/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5233 - accuracy: 0.6190
Epoch 2508/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5233 - accuracy: 0.6190
Epoch 2509/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5233 - accuracy: 0.6190
Epoch 2510/5000
1/1 [==============================] - 0

In [9]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 196ms/step - loss: 0.3587 - accuracy: 0.8333


[0.35872069001197815, 0.8333333134651184]

In [10]:
model.predict(X_test_scaled)

array([[0.770215  ],
       [0.38931814],
       [0.24452779],
       [0.5304952 ],
       [0.78984445],
       [0.88192236]], dtype=float32)

In [11]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: have_insurance, dtype: int64

In [12]:
coef, intercept = model.get_weights()

In [13]:
coef, intercept

(array([[5.72312  ],
        [1.1356844]], dtype=float32), array([-2.6160254], dtype=float32))

In [14]:
def sigmoid(x):
        import math
        return 1 / (1 + math.exp(-x))
sigmoid(18)

0.9999999847700205

In [15]:
X_test

age  affordability
2    47              1
10   18              1
21   26              0
11   28              1
14   49              1
9    61              1

In [16]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

0.7702149960453393

In [17]:
prediction_function(.18, 1)

0.38931812757233897

In [18]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [20]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))                                                              

In [33]:
class myNN:
    def __init__(self):
        pass 
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordability'],y, epochs, loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordability'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
               print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
               print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
               break

               return w1, w2, bias

In [28]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=8000, loss_thresold=0.4631)

Final weights and bias: w1: 7.288712557229623, w2: 1.1747479100391536, bias: -3.245111354981621


In [29]:
customModel.predict(X_test_scaled)

2     0.795000
10    0.318995
21    0.205863
11    0.492620
14    0.817739
9     0.914959
dtype: float64

In [30]:
coef, intercept

(array([[5.72312  ],
        [1.1356844]], dtype=float32), array([-2.6160254], dtype=float32))

In [31]:
X_test_scaled

age  affordability
2   0.47              1
10  0.18              1
21  0.26              0
11  0.28              1
14  0.49              1
9   0.61              1

In [32]:
model.predict(X_test_scaled)

array([[0.770215  ],
       [0.38931814],
       [0.24452779],
       [0.5304952 ],
       [0.78984445],
       [0.88192236]], dtype=float32)